# Overview

# Start database

In [ ]:
%run start_db.ipynb

In [ ]:
engine = sa.create_engine(DB_URL, connect_args={'options': '-csearch_path=University'})

In [ ]:
engine.table_names()

## Load cell magic

In [ ]:
import pandas as pd
import sqlalchemy as sa

In [ ]:
sql_query = """\
select *
from student
limit 2;
"""

df = pd.read_sql_query(sql_query, engine)

display(df)

In [ ]:
%run sql_magic.ipynb

In [ ]:
%%sql
select *
from student
limit 2;

In [ ]:
%%sql
select *
from course
limit 2;

In [ ]:
%%sql
select *
from offering
limit 2;

In [ ]:
%%sql
select *
from took
limit 2;

# Exercises

## GROUP BY and HAVING

#### 1. Write a query to find the average grade, minimum grade, and maximum grade for each offering.

In [ ]:
%%sql
select oid, avg(grade), min(grade), max(grade)
from Took
group by oid;

#### 2. Which of these queries is legal?

In [ ]:
%%sql
SELECT surname, sid
FROM Student, Took
WHERE Student.sid = Took.sid
GROUP BY sid;

In [ ]:
%%sql
SELECT instructor, max(grade),
count(Took.oid)
FROM Took, Offering
WHERE Took.oid = Offering.oid
GROUP BY instructor;

In [ ]:
%%sql
SELECT surname, Student.sid
FROM Student, Took
WHERE Student.sid = Took.sid
GROUP BY campus;

In [ ]:
%%sql
SELECT Course.dept, Course.cnum,
count(oid), count(instructor)
FROM Course, Offering
WHERE Course.dept = Offering.dept and
Course.cnum = Offering.cnum
GROUP BY Course.dept, Course.cnum
ORDER BY count(oid);

#### 3. Find the sid and minimum grade of each student with an average over 80.

In [ ]:
%%sql
SELECT SID, min(grade)
FROM Took
GROUP BY sID
HAVING AVG(grade) > 80;

#### 4. Find the sid, surname, and average grade of each student, but keep the data only for those students who have taken at least 10 courses.

In [ ]:
%%sql
SELECT Student.sID, surname, avg(grade)
FROM Student, Took
WHERE Student.sID = Took.sID
GROUP BY Student.sID
HAVING count(grade) >= 10;

#### 5. For each student who has passed at least 10 courses, report their sid and average grade on the courses that they passed.

In [ ]:
%%sql
SELECT sid, AVG(grade)
FROM took
WHERE grade >= 50
GROUP BY sid
HAVING count(*) >= 10;

#### 6. For each student who has passed at least 10 courses, report their sid and average grade on *all of their courses*.

In [ ]:
%%sql
CREATE OR REPLACE VIEW Seniors AS
SELECT sid
FROM Took
WHERE grade >= 50
GROUP BY sid
HAVING count(*) >= 10;

SELECT Seniors.sid, AVG(grade)
FROM Seniors, Took
WHERE seniors.sid = Took.sid
GROUP BY Seniors.sid;

#### 7. Which of these queries is legal?

In [ ]:
%%sql
SELECT dept
FROM Took, Offering
WHERE Took.oID = Offering.oID
GROUP BY dept
HAVING avg(grade) > 75;

In [ ]:
%%sql
SELECT Took.oID, avg(grade)
FROM Took, Offering
WHERE Took.oID = Offering.oID
GROUP BY Took.oID
HAVING avg(grade) > 75;

In [ ]:
%%sql
SELECT Took.oID, dept, cNum, avg(grade)
FROM Took, Offering
WHERE Took.oID = Offering.oID
GROUP BY Took.oID
HAVING avg(grade) > 75;

In [ ]:
%%sql
SELECT oID, avg(grade)
FROM Took
GROUP BY sID
HAVING avg(grade) > 75;

## Set Operations: Solutions

#### 1. Answer each of the following questions with an arithmetic expression.

Suppose a row occurs $n$ times in table R and $m$ times in table S.

(a) Using bag semantics, how many times will it occur in table R $\cup$ S?

In [ ]:
# solution
# n + m

(b) Using bag semantics, how many times will it occur in table R $\cap$ S?

In [ ]:
# solution
# min(n, m)

(c) Using bag semantics, how many times will it occur in table R $-$ S?

In [ ]:
# solution
# max(n − m, 0)

#### 2. Use a set operation to find all terms when Jepson and Suzuki were both teaching. Include every occurrence of a term from the result of both operands.

In [ ]:
%%sql
(select Term from Offering where instructor = 'Suzuki')
intersect all
(select Term from Offering where instructor = 'Jepson');

#### 3. Find the sID of students who have earned a grade of 85 or more in some course, or who have passed a course taught by Atwood. Ensure that no sID occurs twice in the result.

In [ ]:
%%sql
CREATE OR REPLACE VIEW High as
(select sid from took where grade >= 85);

CREATE OR REPLACE VIEW HighAtwood as
(select sid
from Took, Offering
where Took.oid = Offering.oid and instructor = 'Atwood' and grade >= 50);

(select * from high)
UNION
(select * from highAtwood);

#### 4. Find all terms when csc369 was not offered.

In [ ]:
%%sql
(select term
from Offering)
except
(select term
from Offering
where dept = 'csc' and cNum = 369);